In [377]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [378]:
"""bajaj=pd.read_excel("Bajaj.xlsx")
bank_bazar=pd.read_excel("Bank Bazar.xlsx")
financial=pd.read_excel("Financial Buddha.xlsx")
sbi=pd.read_excel("SBI FAQ.xlsx")"""

In [379]:
"""bajaj.rename(columns={"answer": "Answer"},inplace=True)
bajaj.head()"""

,Unnamed: 0,Question,Answer
0,0,What is an EMI?,Equated Monthly Instalments (EMIs) give you th...
1,1,Are there any foreclosure and part-prepayment ...,"Yes, there are foreclosure and part-prepayment..."
2,2,What is the maximum loan amount you can get wh...,• You can avail an instant personal loan amoun...
3,3,What are the tenor options available?,You can choose a tenor ranging from 12 to 60 m...
4,4,What are the benefits or advantages of taking ...,Please check out the Personal Loan features an...


In [380]:
"""del sbi["Unnamed: 0"]
del bajaj["Unnamed: 0"]
del financial["Unnamed: 0"]
del bank_bazar["Unnamed: 0"]"""

In [381]:
"""#removint /n and /t from 
for i in range(len(sbi["Question"])):
    pre=sbi["Question"][i].replace("\t","")
    sbi["Question"][i]=pre.replace("\n","")
#removing Q. and A.
for i in range(len(sbi["Question"])):
    sbi["Question"][i]=sbi["Question"][i][2:]
    sbi["Answer"][i]=sbi["Answer"][i][2:]"""
    

In [382]:
"""#appending all data
df=bajaj.append(bank_bazar,ignore_index=True)
df=df.append(sbi,ignore_index=True)
df=df.append(financial,ignore_index=True)"""

In [383]:
df=pd.read_excel("Merge.xlsx")

In [384]:
mdf=df

In [385]:
del mdf["Unnamed: 0"]

In [386]:
question=[]
for i in mdf.Question:
    question.append(i)

In [387]:
answer=[]
for i in mdf.Answer:
    answer.append(i)

In [388]:
ps = PorterStemmer()
wordnet=WordNetLemmatizer()
corpusQ = []

for i in range(len(question)):
    txt =  question[i].lower()
    txt = re.sub(r"i'm", "i am", txt)
    txt = re.sub(r"he's", "he is", txt)
    txt = re.sub(r"she's", "she is", txt)
    txt = re.sub(r"that's", "that is", txt)
    txt = re.sub(r"what's", "what is", txt)
    txt = re.sub(r"where's", "where is", txt)
    txt = re.sub(r"\'ll", " will", txt)
    txt = re.sub(r"\'ve", " have", txt)
    txt = re.sub(r"\'re", " are", txt)
    txt = re.sub(r"\'d", " would", txt)
    txt = re.sub(r"won't", "will not", txt)
    txt = re.sub(r"can't", "can not", txt)
    txt = re.sub(r"[^\w\s]", "", txt)
    review = re.sub('[^a-zA-Z]', ' ',txt)
    corpusQ.append(review)
    

In [389]:
corpusQ

['what is an emi',
 'are there any foreclosure and partprepayment charges',
 'what is the maximum loan amount you can get when you apply for a salaried personal loan',
 'what are the tenor options available',
 'what are the benefits or advantages of taking a personal loan',
 'how do i apply for a salaried personal loan',
 'when will i receive the salaried personal loan',
 'how do i repay the loan',
 'what is the online approval process for a salaried personal loan',
 'how do i apply online for an instant personal loan',
 'on what criteria will the loan be sanctioned to me',
 'what are the benefits of applying for a personal loan online',
 'how can i check the status of my loan for which i have applied online',
 'how secure is the information provided by me',
 'what is secure fee for online application',
 'what if i do not pay the secure fees online',
 'how can i pay secure fee',
 'is it safe to use my debit card on this site',
 'how do i cancel a transaction or get the refund',
 'what 

In [390]:
corpusA=[]
for i in range(len(answer)):
    txt =  answer[i].lower()
    txt = re.sub(r"i'm", "i am", txt)
    txt = re.sub(r"he's", "he is", txt)
    txt = re.sub(r"she's", "she is", txt)
    txt = re.sub(r"that's", "that is", txt)
    txt = re.sub(r"what's", "what is", txt)
    txt = re.sub(r"where's", "where is", txt)
    txt = re.sub(r"\'ll", " will", txt)
    txt = re.sub(r"\'ve", " have", txt)
    txt = re.sub(r"\'re", " are", txt)
    txt = re.sub(r"\'d", " would", txt)
    txt = re.sub(r"won't", "will not", txt)
    txt = re.sub(r"can't", "can not", txt)
    txt = re.sub(r"[^\w\s]", "", txt)
    review = re.sub('[^a-zA-Z]', ' ',txt)
    corpusA.append(review)

In [391]:
corpusA

['equated monthly instalments emis give you the ease and benefit of paying back the personal loan amount in smaller convenient instalments the instalments consist of both the principal and interest amounts ',
 'yes there are foreclosure and partprepayment charges applicable to read more on the interest rates and charges click here ',
 ' you can avail an instant personal loan amount up to rs   lakh   however the final amount sanctioned will depend on your income your cibil score and other eligibility criteria ',
 'you can choose a tenor ranging from    to    months depending on your preference and the loan sanctioning conditions',
 'please check out the personal loan features and benefits section for more details ',
 'you can apply online through our online application facility or contact us through various modes mentioned in the contact us section  ',
 'for online applications you will get an instant approval the funds will be disbursed to your account within    hours of document verif

In [392]:
count={}
for line in corpusQ:
    for word in line.split():
        if word not in count:
            count[word]=1
        else:
            count[word]+=1

for line in corpusA:
    for word in line.split():
        if word not in count:
            count[word]=1
        else:
            count[word]+=1

In [393]:
len(count)

1337

In [394]:
for i in range(len(corpusA)):
    corpusA[i]='<SOS> '+corpusA[i]+' <EOS>'

In [395]:
count

{'what': 65,
 'is': 191,
 'an': 35,
 'emi': 34,
 'are': 79,
 'there': 39,
 'any': 62,
 'foreclosure': 18,
 'and': 184,
 'partprepayment': 5,
 'charges': 27,
 'the': 499,
 'maximum': 18,
 'loan': 358,
 'amount': 70,
 'you': 195,
 'can': 148,
 'get': 50,
 'when': 14,
 'apply': 31,
 'for': 194,
 'a': 250,
 'salaried': 12,
 'personal': 120,
 'tenor': 7,
 'options': 5,
 'available': 10,
 'benefits': 5,
 'or': 69,
 'advantages': 5,
 'of': 237,
 'taking': 2,
 'how': 57,
 'do': 43,
 'i': 92,
 'will': 118,
 'receive': 4,
 'repay': 14,
 'online': 36,
 'approval': 15,
 'process': 11,
 'instant': 7,
 'on': 112,
 'criteria': 17,
 'be': 119,
 'sanctioned': 6,
 'to': 272,
 'me': 4,
 'applying': 13,
 'check': 13,
 'status': 19,
 'my': 38,
 'which': 25,
 'have': 50,
 'applied': 5,
 'secure': 10,
 'information': 20,
 'provided': 11,
 'by': 45,
 'fee': 10,
 'application': 34,
 'if': 60,
 'not': 29,
 'pay': 21,
 'fees': 5,
 'it': 63,
 'safe': 3,
 'use': 21,
 'debit': 6,
 'card': 14,
 'this': 53,
 'site': 

In [396]:
vocab={}
i=0
for w,v in count.items():
    vocab[w]=i
    i+=1

In [397]:
len(vocab)

1337

In [398]:
tockens=['<PAD>','<EOS>','<OUT>','<SOS>']
x=len(vocab)
for t in tockens:
    vocab[t]=x
    x+=1

In [399]:
vocab['emi']=vocab['<PAD>']
vocab['<PAD>']=0

In [400]:
vocab

{'what': 0,
 'is': 1,
 'an': 2,
 'emi': 1337,
 'are': 4,
 'there': 5,
 'any': 6,
 'foreclosure': 7,
 'and': 8,
 'partprepayment': 9,
 'charges': 10,
 'the': 11,
 'maximum': 12,
 'loan': 13,
 'amount': 14,
 'you': 15,
 'can': 16,
 'get': 17,
 'when': 18,
 'apply': 19,
 'for': 20,
 'a': 21,
 'salaried': 22,
 'personal': 23,
 'tenor': 24,
 'options': 25,
 'available': 26,
 'benefits': 27,
 'or': 28,
 'advantages': 29,
 'of': 30,
 'taking': 31,
 'how': 32,
 'do': 33,
 'i': 34,
 'will': 35,
 'receive': 36,
 'repay': 37,
 'online': 38,
 'approval': 39,
 'process': 40,
 'instant': 41,
 'on': 42,
 'criteria': 43,
 'be': 44,
 'sanctioned': 45,
 'to': 46,
 'me': 47,
 'applying': 48,
 'check': 49,
 'status': 50,
 'my': 51,
 'which': 52,
 'have': 53,
 'applied': 54,
 'secure': 55,
 'information': 56,
 'provided': 57,
 'by': 58,
 'fee': 59,
 'application': 60,
 'if': 61,
 'not': 62,
 'pay': 63,
 'fees': 64,
 'it': 65,
 'safe': 66,
 'use': 67,
 'debit': 68,
 'card': 69,
 'this': 70,
 'site': 71,
 'c

In [401]:
inv_vocab={w:v for v,w in vocab.items()}

In [402]:
inv_vocab

{0: '<PAD>',
 1: 'is',
 2: 'an',
 1337: 'emi',
 4: 'are',
 5: 'there',
 6: 'any',
 7: 'foreclosure',
 8: 'and',
 9: 'partprepayment',
 10: 'charges',
 11: 'the',
 12: 'maximum',
 13: 'loan',
 14: 'amount',
 15: 'you',
 16: 'can',
 17: 'get',
 18: 'when',
 19: 'apply',
 20: 'for',
 21: 'a',
 22: 'salaried',
 23: 'personal',
 24: 'tenor',
 25: 'options',
 26: 'available',
 27: 'benefits',
 28: 'or',
 29: 'advantages',
 30: 'of',
 31: 'taking',
 32: 'how',
 33: 'do',
 34: 'i',
 35: 'will',
 36: 'receive',
 37: 'repay',
 38: 'online',
 39: 'approval',
 40: 'process',
 41: 'instant',
 42: 'on',
 43: 'criteria',
 44: 'be',
 45: 'sanctioned',
 46: 'to',
 47: 'me',
 48: 'applying',
 49: 'check',
 50: 'status',
 51: 'my',
 52: 'which',
 53: 'have',
 54: 'applied',
 55: 'secure',
 56: 'information',
 57: 'provided',
 58: 'by',
 59: 'fee',
 60: 'application',
 61: 'if',
 62: 'not',
 63: 'pay',
 64: 'fees',
 65: 'it',
 66: 'safe',
 67: 'use',
 68: 'debit',
 69: 'card',
 70: 'this',
 71: 'site',
 7

In [403]:
encoder_inp=[] # for question
for line in corpusQ:
    lst=[]
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
    encoder_inp.append(lst)

In [404]:
encoder_inp

[[0, 1, 2, 1337],
 [4, 5, 6, 7, 8, 9, 10],
 [0, 1, 11, 12, 13, 14, 15, 16, 17, 18, 15, 19, 20, 21, 22, 23, 13],
 [0, 4, 11, 24, 25, 26],
 [0, 4, 11, 27, 28, 29, 30, 31, 21, 23, 13],
 [32, 33, 34, 19, 20, 21, 22, 23, 13],
 [18, 35, 34, 36, 11, 22, 23, 13],
 [32, 33, 34, 37, 11, 13],
 [0, 1, 11, 38, 39, 40, 20, 21, 22, 23, 13],
 [32, 33, 34, 19, 38, 20, 2, 41, 23, 13],
 [42, 0, 43, 35, 11, 13, 44, 45, 46, 47],
 [0, 4, 11, 27, 30, 48, 20, 21, 23, 13, 38],
 [32, 16, 34, 49, 11, 50, 30, 51, 13, 20, 52, 34, 53, 54, 38],
 [32, 55, 1, 11, 56, 57, 58, 47],
 [0, 1, 55, 59, 20, 38, 60],
 [0, 61, 34, 33, 62, 63, 11, 55, 64, 38],
 [32, 16, 34, 63, 55, 59],
 [1, 65, 66, 46, 67, 51, 68, 69, 42, 70, 71],
 [32, 33, 34, 72, 21, 73, 28, 17, 11, 74],
 [0, 4, 75, 30, 76, 77, 13],
 [0, 1, 11, 78, 30, 21, 76, 79, 80, 46, 21, 81, 77, 13],
 [0, 4, 11, 82, 83, 30, 84, 79],
 [0, 35, 44, 11, 85, 86],
 [0, 87, 44, 51, 88, 89, 90],
 [32, 35, 11, 91, 44, 92, 61, 11, 93, 94, 28, 95, 96, 97, 11, 85, 86],
 [87, 11, 91,

In [405]:
decoder_inp=[] # for answer
for line in corpusA:
    lst=[]
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
    decoder_inp.append(lst)

In [406]:
q=[] #13
for i in encoder_inp:
    q.append(len(i))
    

In [407]:
a=[]
for i in decoder_inp:
    a.append(len(i))
    

In [408]:
a.sort()#max 323
print(a)#max 12

[6, 11, 12, 12, 13, 13, 13, 14, 15, 15, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 29, 29, 30, 30, 30, 30, 31, 31, 31, 32, 32, 32, 32, 32, 33, 34, 34, 34, 34, 34, 34, 35, 35, 35, 35, 35, 36, 36, 37, 37, 37, 37, 37, 37, 37, 38, 38, 38, 38, 39, 39, 39, 40, 40, 40, 41, 41, 41, 42, 42, 43, 43, 44, 44, 44, 44, 45, 45, 45, 47, 47, 47, 47, 48, 48, 48, 49, 49, 50, 51, 51, 51, 51, 52, 52, 52, 53, 53, 54, 55, 56, 56, 58, 60, 60, 62, 62, 62, 63, 64, 66, 66, 66, 67, 67, 71, 75, 75, 77, 78, 80, 82, 86, 87, 88, 92, 100, 100, 107, 116, 128, 133, 158, 322]


In [409]:
q.sort()#max 22
print(q[-1])#max 12

22


In [410]:
encoder_inp=pad_sequences(encoder_inp,67,padding='post',truncating='post')
decoder_inp=pad_sequences(decoder_inp,67,padding='post',truncating='post')

In [411]:
encoder_inp[0]


array([   0,    1,    2, 1337,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0])

In [412]:
decoder_final_output=[]
for i in decoder_inp:
    decoder_final_output.append(i[1:])


In [413]:
decoder_final_output=pad_sequences(decoder_final_output,67,padding='post',truncating='post')

In [414]:
from tensorflow.keras.utils import to_categorical
decoder_final_output=to_categorical(decoder_final_output,len(vocab))

In [415]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input

In [416]:
enc_inp = Input(shape=(67, ))
dec_inp = Input(shape=(67, ))


In [417]:
enc_inp

<tf.Tensor 'input_31:0' shape=(None, 67) dtype=float32>

In [418]:
VOCAB_SIZE = len(vocab)
embed = Embedding(VOCAB_SIZE+1, output_dim=50, 
                  input_length=67,
                  trainable=True                  
                  )

In [419]:
enc_embed = embed(enc_inp)
enc_lstm = LSTM(400, return_sequences=True, return_state=True)
enc_op, h, c = enc_lstm(enc_embed)
enc_states = [h, c]


In [420]:
dec_embed = embed(dec_inp)
dec_lstm = LSTM(400, return_sequences=True, return_state=True)
dec_op, _, _ = dec_lstm(dec_embed, initial_state=enc_states)

In [421]:
dense = Dense(VOCAB_SIZE, activation='softmax')

In [422]:
dense_op = dense(dec_op)

In [423]:
model = Model([enc_inp, dec_inp], dense_op)

In [424]:
model.compile(loss='categorical_crossentropy',metrics=['acc'],optimizer='adam')

In [448]:
model.fit([encoder_inp, decoder_inp],decoder_final_output,epochs=800)

Epoch 1/800
7/7 [==============================] - 3s 496ms/step - loss: 0.1530 - acc: 0.9757
Epoch 2/800
7/7 [==============================] - 4s 506ms/step - loss: 0.1523 - acc: 0.9765
Epoch 3/800
7/7 [==============================] - 4s 531ms/step - loss: 0.1510 - acc: 0.9765
Epoch 4/800
7/7 [==============================] - 3s 497ms/step - loss: 0.1491 - acc: 0.9763
Epoch 5/800
7/7 [==============================] - 4s 506ms/step - loss: 0.1483 - acc: 0.9762
Epoch 6/800
7/7 [==============================] - 4s 516ms/step - loss: 0.1472 - acc: 0.9762
Epoch 7/800
7/7 [==============================] - 4s 511ms/step - loss: 0.1461 - acc: 0.9765
Epoch 8/800
7/7 [==============================] - 3s 491ms/step - loss: 0.1453 - acc: 0.9761
Epoch 9/800
7/7 [==============================] - 3s 498ms/step - loss: 0.1449 - acc: 0.9767
Epoch 10/800
7/7 [==============================] - 4s 515ms/step - loss: 0.1449 - acc: 0.9762
Epoch 11/800
7/7 [==============================] - 3s 492m

7/7 [==============================] - 6s 792ms/step - loss: 0.0916 - acc: 0.9813
Epoch 88/800
7/7 [==============================] - 6s 814ms/step - loss: 0.0914 - acc: 0.9814
Epoch 89/800
7/7 [==============================] - 6s 795ms/step - loss: 0.0905 - acc: 0.9813
Epoch 90/800
7/7 [==============================] - 6s 822ms/step - loss: 0.0908 - acc: 0.9812
Epoch 91/800
7/7 [==============================] - 6s 805ms/step - loss: 0.0899 - acc: 0.9810
Epoch 92/800
7/7 [==============================] - 6s 799ms/step - loss: 0.0897 - acc: 0.9810
Epoch 93/800
7/7 [==============================] - 6s 822ms/step - loss: 0.0900 - acc: 0.9813
Epoch 94/800
7/7 [==============================] - 6s 793ms/step - loss: 0.0929 - acc: 0.9812
Epoch 95/800
7/7 [==============================] - 6s 822ms/step - loss: 0.0920 - acc: 0.9810
Epoch 96/800
7/7 [==============================] - 6s 816ms/step - loss: 0.0913 - acc: 0.9810
Epoch 97/800
7/7 [==============================] - 6s 826ms/st

7/7 [==============================] - 6s 794ms/step - loss: 0.0558 - acc: 0.9887
Epoch 173/800
7/7 [==============================] - 6s 796ms/step - loss: 0.0552 - acc: 0.9889
Epoch 174/800
7/7 [==============================] - 6s 820ms/step - loss: 0.0548 - acc: 0.9882
Epoch 175/800
7/7 [==============================] - 6s 794ms/step - loss: 0.0543 - acc: 0.9895
Epoch 176/800
7/7 [==============================] - 6s 811ms/step - loss: 0.0544 - acc: 0.9888
Epoch 177/800
7/7 [==============================] - 6s 796ms/step - loss: 0.0540 - acc: 0.9887
Epoch 178/800
7/7 [==============================] - 6s 800ms/step - loss: 0.0539 - acc: 0.9888
Epoch 179/800
7/7 [==============================] - 6s 818ms/step - loss: 0.0542 - acc: 0.9882
Epoch 180/800
7/7 [==============================] - 6s 788ms/step - loss: 0.0540 - acc: 0.9888
Epoch 181/800
7/7 [==============================] - 6s 815ms/step - loss: 0.0535 - acc: 0.9894
Epoch 182/800
7/7 [==============================] - 6

7/7 [==============================] - 6s 819ms/step - loss: 0.0285 - acc: 0.9960
Epoch 258/800
7/7 [==============================] - 6s 795ms/step - loss: 0.0285 - acc: 0.9954
Epoch 259/800
7/7 [==============================] - 6s 824ms/step - loss: 0.0281 - acc: 0.9963
Epoch 260/800
7/7 [==============================] - 6s 791ms/step - loss: 0.0277 - acc: 0.9960
Epoch 261/800
7/7 [==============================] - 6s 794ms/step - loss: 0.0272 - acc: 0.9964
Epoch 262/800
7/7 [==============================] - 6s 813ms/step - loss: 0.0269 - acc: 0.9965
Epoch 263/800
7/7 [==============================] - 6s 794ms/step - loss: 0.0262 - acc: 0.9964
Epoch 264/800
7/7 [==============================] - 6s 817ms/step - loss: 0.0262 - acc: 0.9965
Epoch 265/800
7/7 [==============================] - 6s 792ms/step - loss: 0.0254 - acc: 0.9970
Epoch 266/800
7/7 [==============================] - 6s 793ms/step - loss: 0.0248 - acc: 0.9972
Epoch 267/800
7/7 [==============================] - 6

7/7 [==============================] - 6s 810ms/step - loss: 0.0119 - acc: 0.9988
Epoch 343/800
7/7 [==============================] - 6s 817ms/step - loss: 0.0118 - acc: 0.9987
Epoch 344/800
7/7 [==============================] - 6s 797ms/step - loss: 0.0117 - acc: 0.9988
Epoch 345/800
7/7 [==============================] - 6s 822ms/step - loss: 0.0114 - acc: 0.9988
Epoch 346/800
7/7 [==============================] - 6s 787ms/step - loss: 0.0113 - acc: 0.9988
Epoch 347/800
7/7 [==============================] - 6s 825ms/step - loss: 0.0112 - acc: 0.9988
Epoch 348/800
7/7 [==============================] - 6s 827ms/step - loss: 0.0111 - acc: 0.9990
Epoch 349/800
7/7 [==============================] - 6s 790ms/step - loss: 0.0110 - acc: 0.9987
Epoch 350/800
7/7 [==============================] - 6s 814ms/step - loss: 0.0111 - acc: 0.9985
Epoch 351/800
7/7 [==============================] - 5s 786ms/step - loss: 0.0109 - acc: 0.9990
Epoch 352/800
7/7 [==============================] - 6

7/7 [==============================] - 6s 788ms/step - loss: 0.0072 - acc: 0.9990
Epoch 428/800
7/7 [==============================] - 6s 815ms/step - loss: 0.0073 - acc: 0.9988
Epoch 429/800
7/7 [==============================] - 6s 797ms/step - loss: 0.0072 - acc: 0.9989
Epoch 430/800
7/7 [==============================] - 6s 790ms/step - loss: 0.0071 - acc: 0.9990
Epoch 431/800
7/7 [==============================] - 6s 822ms/step - loss: 0.0071 - acc: 0.9990
Epoch 432/800
7/7 [==============================] - 6s 796ms/step - loss: 0.0070 - acc: 0.9989
Epoch 433/800
7/7 [==============================] - 6s 823ms/step - loss: 0.0070 - acc: 0.9990
Epoch 434/800
7/7 [==============================] - 6s 793ms/step - loss: 0.0069 - acc: 0.9989
Epoch 435/800
7/7 [==============================] - 6s 794ms/step - loss: 0.0068 - acc: 0.9991
Epoch 436/800
7/7 [==============================] - 6s 817ms/step - loss: 0.0067 - acc: 0.9990
Epoch 437/800
7/7 [==============================] - 6

7/7 [==============================] - 6s 808ms/step - loss: 0.0052 - acc: 0.9988
Epoch 513/800
7/7 [==============================] - 6s 812ms/step - loss: 0.0050 - acc: 0.9988
Epoch 514/800
7/7 [==============================] - 6s 838ms/step - loss: 0.0049 - acc: 0.9989
Epoch 515/800
7/7 [==============================] - 6s 793ms/step - loss: 0.0049 - acc: 0.9989
Epoch 516/800
7/7 [==============================] - 6s 825ms/step - loss: 0.0048 - acc: 0.9991
Epoch 517/800
7/7 [==============================] - 6s 817ms/step - loss: 0.0048 - acc: 0.9992
Epoch 518/800
7/7 [==============================] - 6s 786ms/step - loss: 0.0048 - acc: 0.9992
Epoch 519/800
7/7 [==============================] - 6s 814ms/step - loss: 0.0048 - acc: 0.9990
Epoch 520/800
7/7 [==============================] - 6s 792ms/step - loss: 0.0048 - acc: 0.9991
Epoch 521/800
7/7 [==============================] - 6s 808ms/step - loss: 0.0046 - acc: 0.9990
Epoch 522/800
7/7 [==============================] - 6

7/7 [==============================] - 6s 811ms/step - loss: 0.0038 - acc: 0.9987
Epoch 598/800
7/7 [==============================] - 5s 782ms/step - loss: 0.0038 - acc: 0.9990
Epoch 599/800
7/7 [==============================] - 5s 779ms/step - loss: 0.0038 - acc: 0.9991
Epoch 600/800
7/7 [==============================] - 6s 807ms/step - loss: 0.0036 - acc: 0.9992
Epoch 601/800
7/7 [==============================] - 5s 778ms/step - loss: 0.0037 - acc: 0.9990
Epoch 602/800
7/7 [==============================] - 6s 803ms/step - loss: 0.0036 - acc: 0.9991
Epoch 603/800
7/7 [==============================] - 5s 778ms/step - loss: 0.0035 - acc: 0.9990
Epoch 604/800
7/7 [==============================] - 5s 778ms/step - loss: 0.0036 - acc: 0.9990
Epoch 605/800
7/7 [==============================] - 6s 802ms/step - loss: 0.0035 - acc: 0.9990
Epoch 606/800
7/7 [==============================] - 5s 776ms/step - loss: 0.0035 - acc: 0.9992
Epoch 607/800
7/7 [==============================] - 6

7/7 [==============================] - 6s 787ms/step - loss: 0.0121 - acc: 0.9988
Epoch 683/800
7/7 [==============================] - 6s 820ms/step - loss: 0.0119 - acc: 0.9991
Epoch 684/800
7/7 [==============================] - 6s 796ms/step - loss: 0.0118 - acc: 0.9990
Epoch 685/800
7/7 [==============================] - 6s 822ms/step - loss: 0.0116 - acc: 0.9990
Epoch 686/800
7/7 [==============================] - 6s 800ms/step - loss: 0.0114 - acc: 0.9990
Epoch 687/800
7/7 [==============================] - 5s 780ms/step - loss: 0.0112 - acc: 0.9989
Epoch 688/800
7/7 [==============================] - 6s 817ms/step - loss: 0.0111 - acc: 0.9989
Epoch 689/800
7/7 [==============================] - 5s 785ms/step - loss: 0.0110 - acc: 0.9987
Epoch 690/800
7/7 [==============================] - 6s 810ms/step - loss: 0.0108 - acc: 0.9990
Epoch 691/800
7/7 [==============================] - 6s 817ms/step - loss: 0.0106 - acc: 0.9990
Epoch 692/800
7/7 [==============================] - 6

7/7 [==============================] - 3s 414ms/step - loss: 0.0056 - acc: 0.9991
Epoch 768/800
7/7 [==============================] - 3s 393ms/step - loss: 0.0055 - acc: 0.9992
Epoch 769/800
7/7 [==============================] - 3s 388ms/step - loss: 0.0055 - acc: 0.9991
Epoch 770/800
7/7 [==============================] - 3s 386ms/step - loss: 0.0055 - acc: 0.9991
Epoch 771/800
7/7 [==============================] - 3s 407ms/step - loss: 0.0055 - acc: 0.9990
Epoch 772/800
7/7 [==============================] - 3s 377ms/step - loss: 0.0054 - acc: 0.9990
Epoch 773/800
7/7 [==============================] - 3s 373ms/step - loss: 0.0054 - acc: 0.9991
Epoch 774/800
7/7 [==============================] - 3s 387ms/step - loss: 0.0054 - acc: 0.9990
Epoch 775/800
7/7 [==============================] - 3s 376ms/step - loss: 0.0054 - acc: 0.9991
Epoch 776/800
7/7 [==============================] - 3s 391ms/step - loss: 0.0053 - acc: 0.9990
Epoch 777/800
7/7 [==============================] - 3

## Interfenrence


In [449]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

In [450]:
enc_model = Model([enc_inp], enc_states)

In [451]:
# decoder Model
decoder_state_input_h = Input(shape=(400,))
decoder_state_input_c = Input(shape=(400,))

In [452]:
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [453]:
decoder_outputs, state_h, state_c = dec_lstm(dec_embed , 
                                    initial_state=decoder_states_inputs)

In [454]:
decoder_states = [state_h, state_c]


In [455]:
dec_model = Model([dec_inp]+ decoder_states_inputs,
                                      [decoder_outputs]+ decoder_states)

In [456]:
print("##########################################")
print("#       start chatting ver. 1.0          #")
print("##########################################")

##########################################
#       start chatting ver. 1.0          #
##########################################


In [457]:
import numpy as np
def clean_text(txt):
    txt =  txt.lower()
    txt = re.sub(r"i'm", "i am", txt)
    txt = re.sub(r"he's", "he is", txt)
    txt = re.sub(r"she's", "she is", txt)
    txt = re.sub(r"that's", "that is", txt)
    txt = re.sub(r"what's", "what is", txt)
    txt = re.sub(r"where's", "where is", txt)
    txt = re.sub(r"\'ll", " will", txt)
    txt = re.sub(r"\'ve", " have", txt)
    txt = re.sub(r"\'re", " are", txt)
    txt = re.sub(r"\'d", " would", txt)
    txt = re.sub(r"won't", "will not", txt)
    txt = re.sub(r"can't", "can not", txt)
    txt = re.sub(r"[^\w\s]", "", txt)
    review = re.sub('[^a-zA-Z]', ' ',txt)
    print(review)
    return review

In [459]:

prepro1 = ""
while prepro1 != 'q':
    prepro1  = input("you : ")
    ## prepro1 = "Hello"

    prepro1 = clean_text(prepro1)
    ## prepro1 = "hello"

    prepro = [prepro1]
    ## prepro1 = ["hello"]

    txt = []
    for x in prepro:
        # x = "hello"
        lst = []
        for y in x.split():
            ## y = "hello"
            try:
                lst.append(vocab[y])
                ## vocab['hello'] = 454
            except:
                lst.append(vocab['<OUT>'])
        txt.append(lst)

    ## txt = [[454]]
    txt = pad_sequences(txt, 67, padding='post')

    ## txt = [[454,0,0,0,.........13]]

    stat = enc_model.predict( txt )

    empty_target_seq = np.zeros( ( 1 , 1) )
     ##   empty_target_seq = [0]


    empty_target_seq[0, 0] = vocab['<SOS>']
    ##    empty_target_seq = [255]

    stop_condition = False
    decoded_translation = ''

    while not stop_condition :

        dec_outputs , h, c= dec_model.predict([ empty_target_seq] + stat )
        decoder_concat_input = dense(dec_outputs)
        ## decoder_concat_input = [0.1, 0.2, .4, .0, ...............]

        sampled_word_index = np.argmax( decoder_concat_input[0, -1, :] )
        ## sampled_word_index = [2]

        sampled_word = inv_vocab[sampled_word_index] + ' '

        ## inv_vocab[2] = 'hi'
        ## sampled_word = 'hi '

        if sampled_word != '<EOS> ':
            decoded_translation += sampled_word  

        if sampled_word == '<EOS> ' or len(decoded_translation.split()) > 67:
            stop_condition = True 

        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        ## <SOS> - > hi
        ## hi --> <EOS>
        stat = [h, c]  

    print("chatbot attention : ", decoded_translation )
    print("==============================================")

you : what emi
what emi
chatbot attention :  the turnaround time tat depends on the customer the case the amount and the documentation served by the customer also all positive outcomes of customer profiling is undertaken the credit managers make it sure that it takes days for salaried employed and about days for self employed 
you : Is it possible to transfer a personal loan to someone?
is it possible to transfer a personal loan to someone
chatbot attention :  if the borrower have cosigners with them for the personal loan then that cosigner is equally responsible for the repayments and everything regarding that very particular loan but when it comes to transfer of personal loan to someone else it is not possible in any condition 
you : can i transfer personal loan to someone
can i transfer personal loan to someone
chatbot attention :  the personal loan application form is made available right in the home page of finance buddha site or this can be applied directly through bajaj finance 